In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
gis_server = config['srv']['server']
pip_server = config['srv']['pip']
gis = config['db']['parksgis']
pip = config['db']['pip']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
pip_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
con_string = 'Driver={' + driver + '};Server=' + gis_server +';Database=' + gis + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
gis_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Set the prop id lookup field dictionary

In [6]:
propid_lookup = {'property_evw': 'gispropnum', 
                 'playground_evw': 'omppropid', 
                 'zone_evw': 'omppropid', 
                 'unmapped_gisallsites_evw': 'omppropid',
                 'schoolyard_to_playground_evw': 'gispropnum',
                 'greenstreet_evw': 'omppropid',
                 'golfcourse_evw': 'omppropid',
                 'restrictivedeclarationsite_evw': 'gispropnum',
                 'structure_evw': 'omppropid'}

In [7]:
field_lookup = {'property_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 'jurisdiction', 
                                 'typecategory', 'featurestatus', 'gisobjid'], 
                 'playground_evw': ['parentid', 'omppropid', 'department', 'signname', 'location', 'acres', 
                                    'jurisdiction', 'typecategory', 'featurestatus', 'gisobjid'],  
                 'zone_evw': ['parentid', 'omppropid', 'department', 'sitename', 'location', 'acres', 
                              'jurisdiction', 'featurestatus', 'gisobjid'], 
                 'unmapped_gisallsites_evw': ['gispropnum', 'omppropid', 'department', 'name', 'location', 
                                              'acres', 'jurisdiction'],
                 'schoolyard_to_playground_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 
                                                  'jurisdiction', 'featurestatus'], 
                 'greenstreet_evw': ['gispropnum', 'omppropid', 'department', 'sitename', 'location', 'acres', 
                                    'jurisdiction', 'featurestatus', 'gisobjid'], 
                 'golfcourse_evw': ['gispropnum', 'omppropid', 'department', 'name', 'location', 'acres', 
                                    'jurisdiction', 'featurestatus', 'gisobjid'],
                 'restrictivedeclarationsite_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 
                                                    'jurisdiction', 'featurestatus'],
                 'structure_evw': ['gispropnum', 'omppropid', 'department', 'description', 'location', 
                                   'jurisdiction', 'featurestatus', 'gisobjid']}

In [ ]:
field_lookup = {'property_evw': {'propnum': 'gispropnum',
                                 'prop id': 'gispropnum', 
                                 'borough': 'department',
                                 'ampsdistrict': 'department',
                                 'prop name': 'signname'
                                 'site name': 'signname',
                                 'prop location': 'location',
                                 'site location': 'location',
                                 'acres': 'acres', 
                                 'jurisdiction': 'jurisdiction', 
                                 'typecategory': 'typecategory', 
                                 'featurestatus':'featurestatus', 
                                 'gisobjid':'gisobjid'}, 
                 'playground_evw': {'propnum': 'parentid', 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'signname', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},  
                 'zone_evw': {'propnum': 'parentid', 
                              'prop id': 'omppropid', 
                              'borough': 'department',
                              'ampsdistrict': 'department',
                              'site name': 'sitename', 
                              'site location': 'location', 
                              'acres': 'acres', 
                              'jurisdiction': 'jurisdiction', 
                              'featurestatus':'featurestatus', 
                              'gisobjid':'gisobjid'}, 
                #Need to complete
                 'unmapped_gisallsites_evw': ['gispropnum', 'omppropid', 'department', 'name', 'location', 
                                              'acres', 'jurisdiction'],
                #FINISHED HERE!!!!
                 'schoolyard_to_playground_evw': {'propnum': 'gispropnum',
                                                  'prop id': 'gispropnum', 
                                                  'borough': 'department',
                                                  'ampsdistrict': 'department',
                                                  'prop name': 'signname'
                                                  'site name': 'signname',
                                                  'prop location': 'location',
                                                  'site location': 'location',
                                                  'acres': 'acres', 
                                                  'jurisdiction': 'jurisdiction', 
                                                  'featurestatus':'featurestatus'},
                 'greenstreet_evw': {'propnum': 'gispropnum', 
                                     'prop id': 'omppropid', 
                                     'borough': 'department',
                                     'ampsdistrict': 'department',
                                     'site name': 'sitename', 
                                     'site location': 'location', 
                                     'acres': 'acres', 
                                     'jurisdiction': 'jurisdiction', 
                                     'featurestatus':'featurestatus', 
                                     'gisobjid':'gisobjid'},
                 'golfcourse_evw': {'propnum': 'gispropnum': , 
                                    'prop id': 'omppropid', 
                                    'borough': 'department',
                                    'ampsdistrict': 'department',
                                    'site name': 'name', 
                                    'site location': 'location', 
                                    'acres': 'acres', 
                                    'jurisdiction': 'jurisdiction', 
                                    'featurestatus':'featurestatus', 
                                    'gisobjid':'gisobjid'},
                #Need to complete
                 'restrictivedeclarationsite_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 
                                                    'jurisdiction', 'featurestatus'],
                #Need to complete
                 'structure_evw': ['gispropnum', 'omppropid', 'department', 'description', 'location', 
                                   'jurisdiction', 'featurestatus', 'gisobjid']}

In [8]:
sourcefc_lookup = {'property_evw': 'Property', 
                   'playground_evw': 'Playground', 
                   'zone_evw': 'Zone', 
                   'unmapped_gisallsites_evw': 'Unmapped',
                   'schoolyard_to_playground_evw': 'Schoolyard To Playground',
                   'greenstreet_evw': 'Greenstreet',
                   'golfcourse_evw': 'GolfCourse',
                   'restrictivedeclarationsite_evw': 'RestrictiveDeclarationSite',
                   'structure_evw': 'Structure'}

## Original GIS Data

In [25]:
#Define the tables that will be queried and interacted with
gis_tables = ['property_evw', 'playground_evw', 'zone_evw', 'unmapped_gisallsites_evw', 
              'schoolyard_to_playground_evw', 'greenstreet_evw', 'golfcourse_evw', 
              'restrictivedeclarationsite_evw', 'structure_evw']

In [26]:
#Create the list of SQL Queries
gis_sql_list = ["select objectid, {}, {} as [prop id], '{}' as sourcefc from parksgis.dpr.{}"
                .format(' ,'.join(field_lookup[t]), propid_lookup[t], sourcefc_lookup[t], t) 
                        for t in gis_tables]

In [27]:
#Create a dictionary with sources and dataframes that contain the original data
gis_df_list = {s: pd.read_sql(con = gis_engine, sql = q) for q, s in zip(gis_sql_list, gis_tables)}

## Original PIP Data

In [13]:
#Define the tables that will be queried and interacted with
pip_tables = ['tbl_ref_allsites', 'tbl_ref_allsites_nosync', 'tbl_pip_allsites', 
              'vw_pip_compatible_inspected_sites', 'allsites']

In [14]:
#Create the list of SQL Queries
pip_sql_list = ['select * from accessnewpip.dbo.'+ t for t in pip_tables]

In [17]:
#Create a list of dataframes with the original dat
pip_df_list = {s: pd.read_sql(con = pip_engine, sql = q) for q, s in zip(pip_sql_list, pip_tables)}

## Perform DML Steps

### Delete Records

In [13]:
#Define the common where clause
where = str(r"where objectid like '%3%'")

In [21]:
#Should be able to filter original DFs

In [28]:
gis_df_list['property_evw'].objectid.str.contains('3')

AttributeError: Can only use .str accessor with string values!

In [15]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['delete from parksgis.dpr.{} {}'.format(g.replace('_evw', ''), where) for g in gis_tables]

In [16]:
gis_con = gis_engine.connect()

In [18]:
for q in sql:
    gis_con.execute(q)

In [ ]:
#Create a list of dataframes with the original data
gis_df_post_delete = {s: pd.read_sql(con = gis_engine, sql = q) for q, s in zip(sql, gis_tables)}

### Update Records

In [ ]:
#Define the common where clause
where = str(r"where objectid like '%4%'")

In [ ]:
set_value = str(r"")

In [ ]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['update parksgis.dpr.{} set  {}'.format(g.replace('_evw', ''), where) for g in gis_tables]

### Insert Non-Duplicate Records

In [39]:
import random

In [66]:
#Define the range for generating records
n_gen = range(0, 100)

In [37]:
#Define the borough portion of IDs
boro = ['B','Q','M','R','X'] 

In [68]:
#Combine the borough portion with the formated (padded with leading 0) random numbers
nums = [boro[random.randint(0,4)] + f'{random.randint(0, 999):03}' for i in n_gen]

In [69]:
nums

['B031',
 'Q987',
 'B623',
 'M749',
 'X520',
 'M462',
 'M809',
 'Q353',
 'Q165',
 'B370',
 'R905',
 'X598',
 'X685',
 'Q481',
 'B949',
 'B851',
 'M019',
 'M477',
 'Q699',
 'X483',
 'X843',
 'M170',
 'M663',
 'B356',
 'M900',
 'R527',
 'R218',
 'Q946',
 'B983',
 'R354',
 'R197',
 'Q556',
 'X008',
 'M820',
 'B829',
 'M805',
 'R978',
 'B089',
 'R160',
 'R319',
 'X106',
 'Q021',
 'M430',
 'Q593',
 'X800',
 'R853',
 'B223',
 'B681',
 'R181',
 'M995',
 'M275',
 'Q573',
 'B552',
 'M260',
 'X161',
 'B780',
 'M970',
 'Q512',
 'R311',
 'Q163',
 'X018',
 'M140',
 'X586',
 'M959',
 'R113',
 'Q065',
 'M686',
 'B193',
 'X514',
 'X540',
 'B129',
 'X165',
 'X787',
 'M418',
 'X780',
 'M050',
 'Q447',
 'M505',
 'M495',
 'Q822',
 'X826',
 'Q875',
 'B962',
 'M636',
 'X842',
 'B029',
 'R082',
 'R716',
 'B802',
 'R813',
 'B536',
 'X160',
 'B802',
 'M985',
 'Q449',
 'M960',
 'M505',
 'Q589',
 'Q327',
 'B634']

### Insert Duplicate Records